In [1]:
from copy import deepcopy

class Queue:
    def __init__(self):
        self.queue = []
    
    def put(self,val):
        self.queue.append(val)

    def get(self):
        return self.queue.pop(0)
    
    def empty(self):
        return len(self.queue) == 0
    
    def clear(self):
        self.queue = []
    
    def __str__(self):
        return str(self.queue)

class Grid:
    def __init__(self, width, height):
        self.grid = [[0 for x in range(width)] for y in range(height)]
        self.width = width
        self.height = height
    
    def __str__(self):
        grid_string = ""
        for row in self.grid:
            row_str = ""
            for val in row:
                row_str += str(val)
            grid_string += row_str + "\n"
        return grid_string
     
    """
    Gets the value at the given coordinate
    """
    def get_val(self,x,y):
        if x < 0 or y < 0:
            return None
        try:
            return self.grid[y][x]
        except:
            return None
    
    """
    Get the coordinates which are valid neighbours
    A valid neighbour is a orthogonal coordinate which is an open space
    """
    def get_neighbors(self,xy):
        x,y = xy
        valid = []
        directions = [[1,0],[-1,0],[0,1],[0,-1]]
        for direction in directions:
            if self.get_val(x+direction[0], y+direction[1]) == ".":
                valid.append([x+direction[0], y+direction[1]])
        return valid
        
    """
    Generate the map in accordance to the given algorithm
    """
    def generate_map(self, key):
        for y in range(self.height):
            for x in range(self.width):
                self.grid[y][x] = "." if self.__is_open_space(x,y,key) else "#"
    
    """
    If the value at the given grid coordinate is an open space, return True.
    Otherwize return False
    """
    def __is_open_space(self,x,y,key):
        val1 = x*x + 3*x + 2*x*y + y + y*y
        val1 += key
        binary_val = bin(val1)[2:]
        nr_ones = 0
        for x in binary_val:
            if x == "1":
                nr_ones += 1
        if nr_ones % 2 == 0:
            return True
        else:
            return False
        
    def draw_path(self, path, start = None, goal = None):
        old_grid = deepcopy(self.grid) # Save original state of grid without the path drawn on it
        for p in path:
            x = int(p[0])
            y = int(p[1])
            self.grid[y][x] = "O"
        
        if start != None:
            self.grid[start[1]][start[0]] = "S"
        if goal != None:
            self.grid[goal[1]][goal[0]] = "G"
        
        print(self.__str__())
        self.grid = old_grid

### Implement BFS - Breath First Search
Keep track of the frontier! I.e the nodes we want to visit.  

For each node we visit, find its valid neighbors.
If we haven't visited the neighbor before, add it to our frontier queue and mark that we reached that frontier node from the one we're currently on.

In [2]:
class PathFinder:
    def __init__(self, network):
        self.network = network
    """
    Backtracks from the goal position until we've reached the start, to get the shortest path
    The path taken is stored in an array from start to end position
    """
    def __backtrack(self, reached_via,start,goal):
        prev_node = goal
        shortest_path = []
        while True:
            if prev_node == None:
                break
            shortest_path.insert(0,prev_node)
            prev_node = reached_via[str(prev_node)]
        
        return shortest_path
    
    """
    Don't include the start position when calculating the lenght of the path.
    In part 1 it should not be accounted for in length calc
    """
    def __path_length(self, reached_via, start, node):
        path = self.__backtrack(reached_via, start, node)
        return len(path) - 1
    
    """
    Breath first search https://en.wikipedia.org/wiki/Breadth-first_search
    """
    def BFS(self, start, goal):
        frontier = Queue()
        frontier.put(start)
        reached_via = {}
        reached_via[str(start)] = None

        while not frontier.empty():
            current_node = frontier.get()
            
            if current_node == goal:
                break
            
            neighbors = self.network.get_neighbors(current_node)
            for neighbor in neighbors:
                if str(neighbor) not in reached_via: # Same as checking if the node has been explored already
                    frontier.put(neighbor)
                    reached_via[str(neighbor)] = current_node
                    
        if str(goal) not in reached_via:
            return False

        shortest_path = self.__backtrack(reached_via, start, goal)
        return shortest_path
    
    
    def get_all_reachable(self, start, maxDist):
        frontier = Queue()
        frontier.put(start)
        reached_via = {}
        reached_via[str(start)] = None

        while not frontier.empty():
            current = frontier.get()
            neighbors = self.network.get_neighbors(current)
            
            # Check distance from start to current node
            # If at max distance, no need to check neigbours, continue to next node in frontier
            if self.__path_length(reached_via, start, current) == maxDist:
                continue
            
            for neighbor in neighbors:
                if str(neighbor) not in reached_via:
                    reached_via[str(neighbor)] = current
                    frontier.put(neighbor)
        
        nodes = []
        for node in reached_via.keys():
            a = node.split(", ")
            x = int(a[0][1:])
            y = int(a[1][:-1])
            nodes.append([x,y])
        
        return nodes

In [10]:
def main():
    width = 100
    height = 60
    grid = Grid(width, height)
    grid.generate_map(1352)

    pathfinder = PathFinder(grid)
    
    print("## Part 1")
    start = [1,1]
    goal =[31,39]
    shortest_path = pathfinder.BFS(start, goal)
    if shortest_path:
        steps_taken = len(shortest_path) - 1 # -1 since the start position shouldn't be counted as a step
        print(f"Fewest steps needed to reach {goal} from {start} is: {steps_taken} st\n")
        grid.draw_path(shortest_path, start, goal)
        
    else:
        print("Goal can't be reached")
    
    
    
    print("\n\n## Part 2")
    max_distance = 50
    nodes = pathfinder.get_all_reachable(start, max_distance)
    print(f"Nr of unique nodes with a max distance of {max_distance} away from {start} is {len(nodes)} st\n")
    grid.draw_path(nodes, [1,1], None)
        
        
main()

## Part 1
Fewest steps needed to reach [31, 39] from [1, 1] is: 90 st

.##.##............###.....#####.#####....#..##..#.###....#....#......#..#.##.#....##.###.###..##.##.
#S######.##.####...#.##....#..#.#..#.##.####.##....####.###.#.##.##.###......###........#...#..##.#.
.OOO##.#..#.##.#.#.######..#.##..#.####..#.#######..###..##..#.##.#.##########.#####..#.#.#.......##
##.OOO###......#..#..#..####.###..#..#.#.##..#.......###...#....#....##...#......#####..#..##.###.##
.####O####..###.#.##.#.#...#..#.#.##.##......#..####..#.######.#####....#...##.......#.###..#.###...
.....OO######.###..###..##.#..###..##.#..##.####...#..##....##.#...#.###.######.######.#.##....#####
..##.#OO##..#.......###.#..###......#.#####..#.#...#...#..#....#...#...###....#.##..##.######...##.#
##.#...OOOO.##.##.#.....#....#.##.#.###..#.#.####..##.###########..##.#....##.....#..##..#..##.....#
.##.######O#.#.##...###.##...##.#..#..##.##...#######.....##..#######.#########.#.....##.#.#.#.#####
#.##....##OO##.#.###